In [ ]:
# Single GPU
import sagemaker
from sagemaker.pytorch import PyTorch
from sagemaker.debugger import TensorBoardOutputConfig

role = sagemaker.get_execution_role()
sess = sagemaker.Session(default_bucket="tf-binding-sites")

training_data_s3_path = "s3://tf-binding-sites/pretraining/data/"
output_s3_path = "s3://tf-binding-sites/pretraining/models/results"

tensorboard_output_config = TensorBoardOutputConfig(
    s3_output_path="s3://tf-binding-sites/pretraining/models/results/tensorboard",
    container_local_output_path="/opt/ml/output/tensorboard"
)


estimator = PyTorch(
    base_job_name="pretraining",
    entry_point="pretrain.py",
    source_dir="../training",  # Directory containing training script and other files
    output_path=output_s3_path,
    code_location="s3://tf-binding-sites/pretraining/models/results/code",
    role=role,
    py_version="py310",
    framework_version='2.0.0',
    volume_size=600, 
    instance_count=1,
    instance_type='ml.g5.8xlarge',
    hyperparameters={
        'learning-rate': 1e-5
    },
    tensorboard_output_config=tensorboard_output_config,
)

estimator.fit({'training': training_data_s3_path}, wait=False)

